In [28]:
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.cross_validation import train_test_split
from sklearn.grid_search import GridSearchCV
import matplotlib.pylab as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn import preprocessing
from sklearn.metrics import log_loss
from sklearn.metrics import make_scorer
from sklearn.cross_validation import StratifiedShuffleSplit
from matplotlib.colors import LogNorm
from sklearn.decomposition import PCA
from keras.layers.advanced_activations import PReLU
from keras.layers.core import Dense, Dropout, Activation
from keras.layers.normalization import BatchNormalization
from keras.models import Sequential
from keras.utils import np_utils
from copy import deepcopy
%matplotlib inline  

In [29]:
df_raw=pd.read_csv('/Users/francois-guillaume.rideau/Documents/Kaggle/Kaggle-SF/train.csv')

In [30]:
df_raw=df_raw[['Category','Address']]
df_raw.head()

,Category,Address
0,WARRANTS,OAK ST / LAGUNA ST
1,OTHER OFFENSES,OAK ST / LAGUNA ST
2,OTHER OFFENSES,VANNESS AV / GREENWICH ST
3,LARCENY/THEFT,1500 Block of LOMBARD ST
4,LARCENY/THEFT,100 Block of BRODERICK ST


In [38]:
df_a=df_raw.loc[:,'Address']
type(df_a)

pandas.core.series.Series

In [32]:
print(sum(df_a.apply(lambda x: int(' AV' in x))))
print(sum(df_a.apply(lambda x: int(' BL' in x))))
print(sum(df_a.apply(lambda x: int(' CR' in x))))
print(sum(df_a.apply(lambda x: int(' CT' in x))))
print(sum(df_a.apply(lambda x: int(' DR' in x))))
print(sum(df_a.apply(lambda x: int(' BL' in x))))
print(sum(df_a.apply(lambda x: int(' LN' in x))))
print(sum(df_a.apply(lambda x: int(' PL' in x))))
print(sum(df_a.apply(lambda x: int(' PZ' in x))))
print(sum(df_a.apply(lambda x: int(' RD' in x))))
# print(sum(df_a.apply(lambda x: int(' ST' in x))))
print(sum(df_a.apply(lambda x: int(' TR' in x))))
print(sum(df_a.apply(lambda x: int(' HWY' in x))))
print(sum(df_a.apply(lambda x: int(' HY' in x))))
print(sum(df_a.apply(lambda x: int(' WY' in x))))
print(sum(df_a.apply(lambda x: int(' WAY' in x))))


200999
28751
1811
3285
15611
28751
1936
3392
2350
4120
3258
169
12251
7485
445


In [33]:
sum(df_a.apply(lambda x: int('/' in x)))

260818

In [34]:
df_raw['AV']=df_a.apply(lambda x: int(' AV' in x))
df_raw['BL']=df_a.apply(lambda x: int(' BL' in x))
df_raw['CR']=df_a.apply(lambda x: int(' CR' in x))
df_raw['CT']=df_a.apply(lambda x: int(' CT' in x))
df_raw['DR']=df_a.apply(lambda x: int(' DR' in x))
df_raw['BL']=df_a.apply(lambda x: int(' BL' in x))
df_raw['LN']=df_a.apply(lambda x: int(' LN' in x))
df_raw['PL']=df_a.apply(lambda x: int(' PL' in x))
df_raw['PZ']=df_a.apply(lambda x: int(' PZ' in x))
df_raw['RD']=df_a.apply(lambda x: int(' RD' in x))
df_raw['TR']=df_a.apply(lambda x: int(' TR' in x))
df_raw['HWY']=df_a.apply(lambda x: int((' HY' in x) or ('HWY' in x)) )
df_raw['WAY']=df_a.apply(lambda x: int((' WY' in x) or ('WAY' in x)) )

df_raw['IsIntersect'] = df_a.apply(lambda x: int('/' in x))



In [35]:
df_raw.columns

Index([u'Category', u'Address', u'AV', u'BL', u'CR', u'CT', u'DR', u'LN',
       u'PL', u'PZ', u'RD', u'TR', u'HWY', u'WAY', u'IsIntersect'],
      dtype='object')

In [ ]:
#df_raw['Address']=df_raw['Address'].astype('category')
#sum(df_raw['Address'].value_counts() > 1)
#bool = df_raw['Address'].value_counts() >30
#sum(df_raw['Address'].value_counts()[bool==True])

In [40]:
addresses=sorted(df_raw["Address"].unique())
categories=sorted(df_raw["Category"].unique())
C_counts=df_raw.groupby(["Category"]).size()
A_C_counts=df_raw.groupby(["Address","Category"]).size()

A_counts=df_raw.groupby(["Address"]).size()
A_counts=pd.DataFrame(A_counts)
A_counts.columns=['count']
A_counts.head()


,count
Address,
0 Block of HARRISON ST,1
0 Block of 10TH AV,5
0 Block of 10TH ST,49
0 Block of 11TH ST,48
0 Block of 12TH AV,7


In [47]:
df2 = pd.merge(df_raw,A_counts,left_on='Address',how='left',right_index=True)
df2['one']=1
df2.head()

,Category,Address,AV,BL,CR,CT,DR,LN,PL,PZ,RD,TR,HWY,WAY,IsIntersect,count,one
0,WARRANTS,OAK ST / LAGUNA ST,0,0,0,0,0,0,0,0,0,0,0,0,1,45,1
1,OTHER OFFENSES,OAK ST / LAGUNA ST,0,0,0,0,0,0,0,0,0,0,0,0,1,45,1
2,OTHER OFFENSES,VANNESS AV / GREENWICH ST,1,0,0,0,0,0,0,0,0,0,0,0,1,23,1
3,LARCENY/THEFT,1500 Block of LOMBARD ST,0,0,0,0,0,0,0,0,0,0,0,0,0,155,1
4,LARCENY/THEFT,100 Block of BRODERICK ST,0,0,0,0,0,0,0,0,0,0,0,0,0,52,1


In [59]:
table = pd.pivot_table(df2, values='one',index=['Address'],
                 columns=['Category'], fill_value=0, aggfunc=np.sum)
Total=table.sum(axis=1)

def myfun(v):
    w=v.astype(float)
    return(np.divide(w,Total))
table=table.apply(myfun)
table
table.columns=["proba_"+str(x) for x in range(len(table.columns))]


In [61]:
df3=pd.merge(df2,table,left_on='Address',how='left',right_index=True).drop(['one'],axis=1)
df3

,Category,Address,AV,BL,CR,CT,DR,LN,PL,PZ,...,proba_29,proba_30,proba_31,proba_32,proba_33,proba_34,proba_35,proba_36,proba_37,proba_38
0,WARRANTS,OAK ST / LAGUNA ST,0,0,0,0,0,0,0,0,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.022222,0.088889,0.066667,0.000000
1,OTHER OFFENSES,OAK ST / LAGUNA ST,0,0,0,0,0,0,0,0,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.022222,0.088889,0.066667,0.000000
2,OTHER OFFENSES,VANNESS AV / GREENWICH ST,1,0,0,0,0,0,0,0,...,0.000000,0.000000,0.000000,0.043478,0.0,0.000000,0.000000,0.000000,0.043478,0.000000
3,LARCENY/THEFT,1500 Block of LOMBARD ST,0,0,0,0,0,0,0,0,...,0.000000,0.000000,0.000000,0.025806,0.0,0.019355,0.038710,0.032258,0.019355,0.000000
4,LARCENY/THEFT,100 Block of BRODERICK ST,0,0,0,0,0,0,0,0,...,0.000000,0.000000,0.000000,0.057692,0.0,0.000000,0.096154,0.096154,0.000000,0.019231
5,LARCENY/THEFT,0 Block of TEDDY AV,1,0,0,0,0,0,0,0,...,0.000000,0.000000,0.014706,0.029412,0.0,0.029412,0.073529,0.147059,0.014706,0.014706
6,VEHICLE THEFT,AVALON AV / PERU AV,1,0,0,0,0,0,0,0,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,1.000000,0.000000,0.000000
7,VEHICLE THEFT,KIRKWOOD AV / DONAHUE ST,1,0,0,0,0,0,0,0,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.500000,0.250000,0.000000,0.000000
8,LARCENY/THEFT,600 Block of 47TH AV,1,0,0,0,0,0,0,0,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.142857,0.190476,0.000000,0.000000
9,LARCENY/THEFT,JEFFERSON ST / LEAVENWORTH ST,0,0,0,0,0,0,0,0,...,0.000000,0.000000,0.000000,0.100000,0.0,0.000000,0.066667,0.066667,0.100000,0.000000


In [63]:
len(df3)

878049

In [ ]:
PA=3.0/5
x= (math.log(PA)-math.log(1.-PA))
x

In [64]:
s = pd.Series(list('abc'))

In [66]:
s

0    a
1    b
2    c
dtype: object

In [67]:
'a' in s

False